In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
path = "/content/gdrive/My Drive/classificationText"
os.chdir(path)
!ls

latest_checkpoint.pt	   result.txt  topic_detection_test_unlabel.v1.0.txt
PhoBERT_base_fairseq	   test.ipynb  topic_detection_train.v1.0.txt
PhoBERT_base_transformers  test.txt    vncorenlp


In [3]:
!pip install transformers

     |████████████████████████████████| 1.4MB 15.9MB/s 
     |████████████████████████████████| 890kB 47.9MB/s 
     |████████████████████████████████| 2.9MB 49.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=39140d286192be5930889a8e800358fe719101bdc03236611fde1c9742f52d09
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
!pip install fastBPE
!pip install fairseq

  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp36-cp36m-linux_x86_64.whl size=481511 sha256=1e4c78fb4730db1b2635fce352cd31302b336c95f38123ddd1b30f730eeff58e
  Stored in directory: /root/.cache/pip/wheels/f3/0c/9c/fc62058b4d473a5602bcd3d3edfece796f123875379ea82d79
Successfully built fastBPE
     |████████████████████████████████| 1.7MB 19.1MB/s 
     |████████████████████████████████| 71kB 10.1MB/s 
     |████████████████████████████████| 122kB 31.5MB/s 
     |████████████████████████████████| 112kB 51.6MB/s 
     |████████████████████████████████| 276kB 48.8MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-cp36-none-any.whl size=141230 sha256=7b279911a043e0f6a815d6c590134effdcf48326c3a6833c3019b41318613a3a
  Stored in directory: /root/.cache/pip/wheels/e3/e2/fa/b78480b448b8579ddf393bebd3f47ee23aa84c89b6a78285c8
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=b7339315c3c99b3716a55974dcaffc32ac868e

In [5]:
!pip install vncorenlp
# !mkdir -p vncorenlp/models/wordsegmenter
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
# !mv VnCoreNLP-1.1.1.jar vncorenlp/ 
# !mv vi-vocab vncorenlp/models/wordsegmenter/
# !mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

     |████████████████████████████████| 2.7MB 13.5MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-cp36-none-any.whl size=2645934 sha256=1a9af0d8532c13e2f598e33fea3357734179b3cf4894cb2dd393a5130d1e6321
  Stored in directory: /root/.cache/pip/wheels/09/54/8b/043667de6091d06a381d7745f44174504a9a4a56ecc9380c54
Successfully built vncorenlp


In [6]:
from vncorenlp import VnCoreNLP
import regex as re
rdrsegmenter = VnCoreNLP("/content/gdrive/MyDrive/classificationText/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

text = "Đại học Công Nghệ Hà Nội"
word_segmented_text = rdrsegmenter.tokenize(text) 
print(word_segmented_text)

[['Đại_học', 'Công_Nghệ', 'Hà_Nội']]


In [ ]:
# !wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz
# !tar -xzvf PhoBERT_base_transformers.tar.gz

In [ ]:
# !wget https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
# !tar -xzvf PhoBERT_base_fairseq.tar.gz

In [7]:
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--bpe-codes', 
    default="/content/gdrive/MyDrive/classificationText/PhoBERT_base_transformers/bpe.codes",
    required=False,
    type=str,
    help='path to fastBPE BPE'
)
args, unknown = parser.parse_known_args()
bpe = fastBPE(args)

# Load the dictionary
vocab = Dictionary()
vocab.add_from_file("/content/gdrive/MyDrive/classificationText/PhoBERT_base_transformers/dict.txt")

In [8]:
vocab.encode_line('<s> ' + 'Nhóm 12 báo cáo bài tập lớn !' + ' </s>')

tensor([   0, 2131,  445,  441, 9866,  387,  834,  103,  381,    2,    2],
       dtype=torch.int32)

In [9]:
def process_text(document):
  document = document.lower() # đưa về chữ thường
  # document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
  document = re.sub(r'\s+', ' ', document).strip() #xóa khoảng trắng thưaf
  return document 

In [10]:
train_text=[]
train_labels=[]
with open("topic_detection_train.v1.0.txt",'r',encoding = 'utf-8') as f:
    str = f.read()
    str = str.split('\n')
    for sentence in str:
        i = sentence.find(" ")
        lbs=sentence[9:i]
        title=sentence[i + 1:-1]
        train_labels.append(lbs)
        train_text.append(process_text(title))
train_labels.remove('')
train_text.pop(-1)
print(len(train_text))
print(len(train_labels))

16000
16000


In [11]:
train_text[8]

'miền đất xa mạc – dubai 🌞🌞🌞 ===================== sự mênh mông của sa mạc ở dubai sẽ khiến cho bạn cảm thấy mình nhỏ bé và lọt thỏm giữa vẻ kỳ vỹ của thiên nhiên, bạn sẽ cứ mải miết đắm chìm trong vẻ đẹp tưởng chừng như ảo ảnh đó. trên lưng lạc đà trên xa mạc 🐪🐪🐪, bạn sẽ có cảm giác mình đang đi trong câu chuyện cổ alibaba hay trở thành một thương nhân trên con đường tơ lụa huyền thoại nối liền đông tây. bạn cũng được trải nghiểm hành trình đua xe sa mạc bằng xe landcruiser. lên tầng 124 của tào tháp 164 tầng cao nhất thế giới để chiêm ngương xa xỉ của dubai🔬🔬🔬. hãy để hanoitourist giúp bạn thực hiện điều đó để trải nghiệm những cảm giác chưa từng có. 💖💖💖 ===================== dubai – vương quốc xa xỉ bay: emirates 5* khởi hành: 12/03, 21/03, 02/04, 19/04, 08/05, 21/05, 04/06, 20/06 thời gian: 6 ngày/5 đêm giá: 24.900.000đ chi tiết: http://bit.ly/dubai-hanoitourist ===================== 👌 liên hệ hanoitourist 👌 📩 địa chỉ: số 18 lý thường kiệt, hoàn kiếm, hà nội ☎ hotline: 04 62703307 

In [12]:
train_labels[:5]

['Du_lich', 'Nha_dat', 'Nha_dat', 'Nha_dat', 'Mua_sam']

In [13]:
list_labels=list(set(train_labels))
print(list_labels)

['Mua_sam', 'Mang_internet_va_vien_thong', 'Nha_va_vuon', 'Giao_thong', 'Giao_duc', 'Suc_khoe_va_benh_tat', 'Du_lich', 'Kinh_doanh_va_Cong_nghiep', 'Chinh_tri', 'Lam_dep_va_the_hinh', 'Khoa_hoc', 'Nghe_thuat', 'May_tinh_va_thiet_bi_dien_tu', 'Cong_nghe_moi', 'Giai_tri', 'Tai_chinh', 'Sach', 'Nha_dat', 'Phap_luat', 'The_thao', 'Do_an_va_do_uong', 'Thoi_quen_va_so_thich', 'Con_nguoi_va_xa_hoi']


In [16]:
for lb in list_labels:
  print(lb," : ",train_labels.count(lb))


Mua_sam  :  1169
Mang_internet_va_vien_thong  :  593
Nha_va_vuon  :  262
Giao_thong  :  108
Giao_duc  :  697
Suc_khoe_va_benh_tat  :  194
Du_lich  :  881
Kinh_doanh_va_Cong_nghiep  :  2356
Chinh_tri  :  796
Lam_dep_va_the_hinh  :  283
Khoa_hoc  :  160
Nghe_thuat  :  609
May_tinh_va_thiet_bi_dien_tu  :  202
Cong_nghe_moi  :  33
Giai_tri  :  211
Tai_chinh  :  1379
Sach  :  415
Nha_dat  :  2542
Phap_luat  :  131
The_thao  :  76
Do_an_va_do_uong  :  2356
Thoi_quen_va_so_thich  :  160
Con_nguoi_va_xa_hoi  :  387


In [14]:
len(list_labels)

23

In [ ]:
dict_label={}
for i in range(len(list_labels)):
  dict_label[list_labels[i]]=i
dict_label

{'Chinh_tri': 20,
 'Con_nguoi_va_xa_hoi': 22,
 'Cong_nghe_moi': 14,
 'Do_an_va_do_uong': 18,
 'Du_lich': 4,
 'Giai_tri': 11,
 'Giao_duc': 8,
 'Giao_thong': 12,
 'Khoa_hoc': 10,
 'Kinh_doanh_va_Cong_nghiep': 3,
 'Lam_dep_va_the_hinh': 16,
 'Mang_internet_va_vien_thong': 21,
 'May_tinh_va_thiet_bi_dien_tu': 6,
 'Mua_sam': 7,
 'Nghe_thuat': 0,
 'Nha_dat': 19,
 'Nha_va_vuon': 15,
 'Phap_luat': 17,
 'Sach': 1,
 'Suc_khoe_va_benh_tat': 5,
 'Tai_chinh': 9,
 'The_thao': 2,
 'Thoi_quen_va_so_thich': 13}

In [ ]:
for i in range(len(train_labels)):
  train_labels[i]= dict_label[train_labels[i]]


In [ ]:
train_labels[:5]

[4, 19, 19, 19, 7]

In [ ]:
from sklearn.model_selection import train_test_split

train_sents, val_sents, train_labels, val_labels = train_test_split(train_text, train_labels, test_size=0.1)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_LEN = 125

train_ids = []
for sent in train_sents:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    train_ids.append(encoded_sent)

val_ids = []
for sent in val_sents:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    val_ids.append(encoded_sent)
    
train_ids = pad_sequences(train_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
val_ids = pad_sequences(val_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")

In [ ]:
train_masks = []
for sent in train_ids:
    mask = [int(token_id > 0) for token_id in sent]
    train_masks.append(mask)

val_masks = []
for sent in val_ids:
    mask = [int(token_id > 0) for token_id in sent]

    val_masks.append(mask)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch

train_inputs = torch.tensor(train_ids)
val_inputs = torch.tensor(val_ids)
train_labels = torch.tensor(train_labels)

val_labels = torch.tensor(val_labels)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=32)

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW

config = RobertaConfig.from_pretrained(
    "/content/gdrive/MyDrive/classificationText/PhoBERT_base_transformers/config.json", from_tf=False, num_labels = 23, output_hidden_states=False,
)
BERT_SA = RobertaForSequenceClassification.from_pretrained(
    "/content/gdrive/MyDrive/classificationText/PhoBERT_base_transformers/model.bin",
    config=config
)


Some weights of the model checkpoint at /content/gdrive/MyDrive/classificationText/PhoBERT_base_transformers/model.bin were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model ch

In [ ]:
BERT_SA.cuda()
print('Done')

Done


BertForSequenceClassification sẽ nhận đầu vào là input_ids và input_mask, đầu ra trả về luôn loss cho classification task và phân phối xác xuất do mô hình dự đoán đầu ra

In [ ]:
BERT_SA

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=0)
      (position_embeddings): Embedding(258, 768, padding_idx=0)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    F1_score = f1_score(pred_flat, labels_flat, average='macro')
    
    return accuracy_score(pred_flat, labels_flat), F1_score

In [ ]:
import random
from tqdm import tqdm_notebook
device = 'cuda'
epochs = 10

param_optimizer = list(BERT_SA.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5, correct_bias=False)


for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    total_loss = 0
    BERT_SA.train()
    train_accuracy = 0
    nb_train_steps = 0
    train_f1 = 0
    
    for step, batch in tqdm_notebook(enumerate(train_dataloader)):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        BERT_SA.zero_grad()
        outputs = BERT_SA(b_input_ids, 
            token_type_ids=None, 
            attention_mask=b_input_mask, 
            labels=b_labels)
        loss = outputs[0]
        total_loss += loss.item()
        
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        tmp_train_accuracy, tmp_train_f1 = flat_accuracy(logits, label_ids)
        train_accuracy += tmp_train_accuracy
        train_f1 += tmp_train_f1
        nb_train_steps += 1
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(BERT_SA.parameters(), 1.0)
        optimizer.step()
        
    avg_train_loss = total_loss / len(train_dataloader)
    print(" Accuracy: {0:.4f}".format(train_accuracy/nb_train_steps))
    print(" F1 score: {0:.4f}".format(train_f1/nb_train_steps))
    print(" Average training loss: {0:.4f}".format(avg_train_loss))

    print("Running Validation...")
    BERT_SA.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    eval_f1 = 0
    for batch in tqdm_notebook(val_dataloader):

        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = BERT_SA(b_input_ids, 
            token_type_ids=None, 
            attention_mask=b_input_mask)
            logits = outputs[0]
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            tmp_eval_accuracy, tmp_eval_f1 = flat_accuracy(logits, label_ids)

            eval_accuracy += tmp_eval_accuracy
            eval_f1 += tmp_eval_f1
            nb_eval_steps += 1
    print(" Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
    print(" F1 score: {0:.4f}".format(eval_f1/nb_eval_steps))
print("Training complete!")

======== Epoch 1 / 10 ========
Training...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



 Accuracy: 0.7758
 F1 score: 0.6589
 Average training loss: 0.8858
Running Validation...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:60: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



 Accuracy: 0.8675
 F1 score: 0.8383
======== Epoch 2 / 10 ========
Training...


...


 Accuracy: 0.8675
 F1 score: 0.8678
======== Epoch 9 / 10 ========
Training...



 Accuracy: 0.9281
 F1 score: 0.9379
 Average training loss: 0.1423
Running Validation...



 Accuracy: 0.8606
 F1 score: 0.8708
======== Epoch 10 / 10 ========
Training...



 Accuracy: 0.9297
 F1 score: 0.9419
 Average training loss: 0.1386
Running Validation...



 Accuracy: 0.8581
 F1 score: 0.8656
Training complete!


In [ ]:
torch.save(BERT_SA,"latest_checkpoint.pt")

#load model
# BERT_SA = torch.load('latest_checkpoint.pt')

In [ ]:
def convertNumber2String(val,dict_label):
  for key, value in dict_label.items(): 
         if val == value: 
             return key 

In [ ]:
# test=["Cho thuê nhà riêng dt 60m/sàn.  Có 4 phòng ngủ. 1p khách thoáng mát an ninh tốt.  Nhà ngõ 401 cổ nhuế gần đại học mỏ .hv tài chính hn. Lh. 01694997361.A Hoàng","mình cần tìm 1 phòng cho khoảng 3 người  quanh khu vực hồ tùng mậu. phòng khép kín có điều hòa .Mng nếu quen ai cho thuê thì gthieu giúp mình . mình cảm ơn !! Nếu hợp lí thì mình chuyển luôn trong chủ nhật hoặc thứ 2"]

In [ ]:
# test.append("Chúc mừng sinh nhật Hoàng Gia Media Group tròn 20 tuổi (28/3/1998 - 28/3/2018) là đơn vị thực hiện sản xuất chương trình CEO-Chìa khóa thành công hơn 13 năm qua.  Được thành lập năm 1998 với tiền thân là Trung tâm Quảng cáo - Tiếp thị - Hội chợ TAF. Ngày nay, Hoanggia Media Group (HMG) nổi tiếng là nhà tư vấn chiến lược thương hiệu và truyền thông chuyên nghiệp. Đồng thời là nhà sản xuất nội dung hàng đầu với các dòng sản phẩm: truyền hình, thương hiệu và truyền thông xã hội đẳng cấp cao.   Trong lĩnh vực truyền thông xã hội, HMG là đối tác tin cậy của nhiều đài truyền hình, cơ quan báo chí trên cả nước. Là đối tác đặc biệt của VTV, HMG đã sản xuất và đưa lên sóng những chương trình truyền hình giàu trí tuệ và giá trị xã hội như: Camera Công sở 2009 -2011, CEO - Chìa khóa thành công CEO từ 2005 đến nay.  Chúc cho Hoàng Gia Media Group ngày càng phát triển, thành công bền vững và luôn sát cánh cùng CEO-Chìa khóa thành công trong chặng đường phía trước cùng cộng đồng Doanh nhân, doanh nghiệp Việt Nam.")

In [ ]:
# test.append("LÀN MÔI XINH TƯƠI CẢ NGÀY CÙNG SON MÔI OHUI. ❤ ❤ ❤Dưỡng , làm mềm môi , trị - chống thâm , đặc biệt rất lâu trôi có thể ăn uống thoả thích ❤ Giá bán : 1 bảng 320k , 2 bảng giá chỉ 600k , mua 5 tặng 1. ❤ MIỄN PHÍ SHIP HÀNG ❤CHỈ THU TIỀN KHÁCH NHẬN ĐỦ HÀNG ❤Son môi có màu sắc tươi tắn trẻ trung , phản xạ ánh sáng tốt , lên môi cực chuẩn màu ( kể cả những bạn có làn môi thâm tối màu nhất ) . ❤Không chỉ có vậy , với khả năng duy trì màu liên tục đến 12h đồng hồ , ❤Và duy trì làn môi ẩm mịn suốt 6h với thành phần chăm sóc môi ẩm mượt và khỏe mạnh. ❤Hình thức mua hàng : 1. Nhắn tin vào hộp thư tin nhắn trang để đặt hàng. 2. Liên hệ : 0942991862 ( zalo , viber )")

In [ ]:
test=[]
with open('topic_detection_test_unlabel.v1.0.txt', 'r',encoding="utf8") as f:
    str = f.read()
    str = str.split('\n')
    test=str
test.remove('')

In [ ]:
len(test)

10017

In [ ]:
test_ids = []
for sent in test:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    test_ids.append(encoded_sent)

test_ids = pad_sequences(test_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
test_masks = []
for sent in test_ids:
    mask = [int(token_id > 0) for token_id in sent]

    test_masks.append(mask)

test_inputs = torch.tensor(test_ids)
test_masks = torch.tensor(test_masks)
test_data = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler,batch_size=32)

print("Running test...")
# BERT_SA.eval()
predict=[]
for batch in tqdm_notebook(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask = batch
    with torch.no_grad():
        outputs = BERT_SA(b_input_ids, 
        token_type_ids=None, 
        attention_mask=b_input_mask)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        pred = np.argmax(logits, axis=1).flatten().tolist()
        for i in range(len(pred)):
          pred[i] = convertNumber2String(pred[i],dict_label)
          predict.append(pred[i])

print(len(predict))

Running test...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



10017


In [ ]:
predict

['Nha_dat',
 'Mang_internet_va_vien_thong',
 'Nha_dat',
 'Sach',
 'Nha_dat',
 'Nghe_thuat',
 'Chinh_tri',
 'Nha_dat',
 'Giao_duc',
 'Sach',
 'Sach',
 'Do_an_va_do_uong',
 'Nha_dat',
 'May_tinh_va_thiet_bi_dien_tu',
 'Nha_dat',
 'Nha_dat',
 'Kinh_doanh_va_Cong_nghiep',
 'Do_an_va_do_uong',
 'Kinh_doanh_va_Cong_nghiep',
 'Mang_internet_va_vien_thong',
 'Giao_duc',
 'May_tinh_va_thiet_bi_dien_tu',
 'Chinh_tri',
 'Mua_sam',
 'Lam_dep_va_the_hinh',
 'Kinh_doanh_va_Cong_nghiep',
 'Sach',
 'Tai_chinh',
 'Do_an_va_do_uong',
 'Mua_sam',
 'Do_an_va_do_uong',
 'Do_an_va_do_uong',
 'Chinh_tri',
 'The_thao',
 'Nghe_thuat',
 'Nha_dat',
 'Cong_nghe_moi',
 'Giao_duc',
 'Nha_dat',
 'Giao_duc',
 'Giao_duc',
 'Kinh_doanh_va_Cong_nghiep',
 'Kinh_doanh_va_Cong_nghiep',
 'Chinh_tri',
 'Tai_chinh',
 'Mua_sam',
 'Nha_dat',
 'Nghe_thuat',
 'Mua_sam',
 'Chinh_tri',
 'The_thao',
 'Do_an_va_do_uong',
 'Tai_chinh',
 'Do_an_va_do_uong',
 'Kinh_doanh_va_Cong_nghiep',
 'Du_lich',
 'Nghe_thuat',
 'Nha_dat',
 'Tai_chin

In [ ]:
with open("result.txt",'w',encoding = 'utf-8') as f:
 for i in range(len(predict)):
     predict[i]=predict[i]+"\n"
     f.write(predict[i])